In [4]:
import pandas as pd
import numpy as np
#一 数据收集 start 
train=pd.read_csv('train.csv',index_col=0)
test=pd.read_csv('test.csv',index_col=0)

In [5]:
#二 数据预处理 start 
print(train['Attrition'].value_counts())
train['Attrition']=train['Attrition'].map(lambda x:1 if x=='Yes' else 0)

No     988
Yes    188
Name: Attrition, dtype: int64


In [6]:
#train.isna().sum()#查看是否有空值 Step1，对数据进行探索 
#Step2，去掉无用特征，处理分类特征
train= train.drop(['EmployeeNumber', 'StandardHours'], axis=1)
test=test.drop(['EmployeeNumber', 'StandardHours'], axis=1)


In [7]:
#分类特征进行特征值编码：
from sklearn import preprocessing 

attr=['BusinessTravel','Department','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime']
lbe_list=[]
for fea in attr:
    lbe=preprocessing.LabelEncoder()
    train[fea]=lbe.fit_transform(train[fea])
    test[fea]=lbe.transform(test[fea])
    lbe_list.append(lbe)

In [23]:
#Step3，模型参数配置x
param = {'boosting_type':'gbdt',
                         'objective' : 'binary', #
                         #'metric' : 'binary_logloss',
                         'metric' : 'auc',
#                          'metric' : 'self_metric',
                         'learning_rate' : 0.01,
                         'max_depth' : 10,
                         'feature_fraction':0.8,
                         'bagging_fraction': 0.9,
                         'bagging_freq': 8,
                         'lambda_l1': 0.6,
                         'lambda_l2': 0,
#                          'scale_pos_weight':k,
#                         'is_unbalance':True
        }

In [24]:
#Step4，模型训练，得出预测结果
from sklearn.model_selection import  train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score
X_train,X_valid,Y_train,Y_valid=train_test_split(train.drop('Attrition',axis=1),train['Attrition'],test_size=0.2,random_state=42)
train_data=lgb.Dataset(X_train,label=Y_train)
valid_data=lgb.Dataset(X_valid,label=Y_valid)
model=lgb.train(param,train_data,1000,[train_data,valid_data],early_stopping_rounds=200,categorical_feature=attr,verbose_eval=25)


D:\anocconda3-5.20\lib\site-packages\lightgbm\basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'Over18', 'OverTime']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 200 rounds
[25]	training's auc: 0.920872	valid_1's auc: 0.781349
[50]	training's auc: 0.941184	valid_1's auc: 0.785769
[75]	training's auc: 0.953125	valid_1's auc: 0.797446
[100]	training's auc: 0.960939	valid_1's auc: 0.807159
[125]	training's auc: 0.970558	valid_1's auc: 0.81469
[150]	training's auc: 0.978022	valid_1's auc: 0.818618
[175]	training's auc: 0.983815	valid_1's auc: 0.823311
[200]	training's auc: 0.988387	valid_1's auc: 0.827677
[225]	training's auc: 0.992105	valid_1's auc: 0.82975
[250]	training's auc: 0.994072	valid_1's auc: 0.827786
[275]	training's auc: 0.995716	valid_1's auc: 0.829204
[300]	training's auc: 0.996803	valid_1's auc: 0.830077
[325]	training's auc: 0.997701	valid_1's auc: 0.830405
[350]	training's auc: 0.998491	valid_1's auc: 0.830623
[375]	training's auc: 0.99894	valid_1's auc: 0.832588
[400]	training's auc: 0.999326	valid_1's auc: 0.834552
[425]	training's auc: 0.999587	valid_1's auc: 0.834989
[450]	tra

In [10]:
predict=model.predict(test)
#predict_y=model.predict(X_valid)
#print('LR准确率: %0.4lf' % accuracy_score(predic_y, Y_valid))
#print(predict)
test['Attrition']=predict
#test['Attrition']=test['Attrition'].map(lambda x:1 if x>=0.5 else 0)
#test['Attrition']=predict
test[['Attrition']].to_csv('Submission.csv')

In [12]:
LightGBM?

Object `LightGBM` not found.
